In [1]:
import os
import glob
import pandas as pd
from pathlib import Path
from scipy import stats
from sklearn.metrics import balanced_accuracy_score
import numpy as np

In [5]:
def run_cv_analysis(base_path):
    paths = list(filter(lambda x: x is not None, [path if path.is_dir() else None for path in\
         Path(base_path).iterdir()]))

    file = pd.read_csv(paths[0] / 'pred.csv').file

    cols = ["file"]
    cols.extend([f"prognosis_{ind}"for ind in range(len(list(paths)))])
    preds = pd.DataFrame(columns=cols)

    preds["file"] = file

    for ind, path_dir in enumerate(paths):
        pred = pd.read_csv(path_dir / 'pred.csv')
        preds[f"prognosis_{ind}"] = pred.prognosis
        
    modes = stats.mode(preds.values[:, 1:], axis=1)[0]
    preds["mode_prognosis"] = modes
    
    preds.to_csv(os.path.join(base_path, 'mode_preds.csv'),
             index=False, columns=["file", "mode_prognosis"])

    preds['mode_prognosis'].replace('SEVERE', 1, inplace=True)
    preds['mode_prognosis'].replace('MILD', 0, inplace=True)

    for ind, path_dir in enumerate(paths):
        preds[f"prognosis_{ind}"].replace('SEVERE', 1, inplace=True)
        preds[f"prognosis_{ind}"].replace('MILD', 0, inplace=True)
        
    test_raw_df = pd.read_excel('/mnt/ncshare/ai4covid_hackathon/raw_data/completeTestClinData.xls')
    test_raw_df = test_raw_df[ ['ImageFile', 'Prognosis'] ]
    test_raw_df.Prognosis.replace('SEVERE', 1, inplace=True)
    test_raw_df.Prognosis.replace('MILD', 0, inplace=True)
    test_raw_df.head()
    
    scores = []

    for ind, path_dir in enumerate(paths):
        scores.append(balanced_accuracy_score(test_raw_df.Prognosis, preds[f"prognosis_{ind}"]))
        
    bs_scores = []
    for _ in range(1_000): # 1000 bootstraps
        indicies = np.random.choice(list(range(len(test_raw_df.Prognosis.values))),
                                   size=len(test_raw_df.Prognosis.values), replace=True)
        bs_scores.append(balanced_accuracy_score(test_raw_df.Prognosis.values[indicies],
                                                preds["mode_prognosis"].values[indicies]))

    print('Balanced accuracy score (CV):\t', np.mean(scores), '+/-', np.std(scores))
    print('Balanced accuracy score (mode):\t', np.mean(bs_scores), '+/-', np.std(bs_scores))

In [6]:
## ORIGINAL COMPETITION SUBMISSION --> trained on full training set, many more models to reduce variance in pred.
run_cv_analysis('/mnt/ncshare/ai4covid_hackathon/raw_output/checkpoints/ENSEMBLE/')

Balanced accuracy score (CV):	 0.7078022875816994 +/- 0.021456113431867808
Balanced accuracy score (mode):	 0.7494314737289155 +/- 0.020599515014316942


In [7]:
## PAPER CHKPT, with early stopping enabled --> on a single model basis, better than the first one
run_cv_analysis('/mnt/ncshare/ai4covid_hackathon/raw_output/checkpoints/CROSS_VAL_paper_v1_w_ES/')

Balanced accuracy score (CV):	 0.7243464052287582 +/- 0.017661577179603757
Balanced accuracy score (mode):	 0.7342137307936832 +/- 0.01978346874623485
